In [1]:
import sys
sys.path.append("../../../")

In [2]:
from onsager import crystal, supercell, cluster
import numpy as np
import scipy as sp
import itertools
import Transitions
import Cluster_Expansion
import MC_JIT
import time
from tqdm import tqdm
from numba import jit, int64, float64

/home/soham/.local/lib/python3.6/site-packages/numba/core/errors.py:144: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)
/usr/local/lib/python3.6/dist-packages/onsager-1.3.3.1-py3.6.egg/onsager/cluster.py:804: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.


In [3]:
# Import ML modules here

In [4]:
# Set up the neural network here 

## Set up the vector cluster expander

In [3]:
import pickle

# See if a cluster expander already exists
try:
    with open("ClusterExpander.pkl", "rb") as fl:
        print("Pre-created Cluster expander found")
        VclusExp = pickle.load(fl)

# Otherwise make one
except:
    N_units = 8 # No. of unit cells along each axis in the supercell
    NSpec = 3
    MaxOrder = 3
    MaxOrderTrans = 3

    a0 = 1.0
    cut = 1.01*a0*np.sqrt(3)/2  # Nearest neighbor cutoff

    crys = crystal.Crystal.BCC(a0, chemistry="A")
    jnetBCC = crys.jumpnetwork(0, cut)

    superlatt = N_units * np.eye(3, dtype=int)
    superBCC = supercell.ClusterSupercell(crys, superlatt)
    # Since the "spectator" tuple in the argument is left empty, all
    # sites of chemistry 0 will be mobile sites.

    vacsite = cluster.ClusterSite((0, 0), np.zeros(3, dtype=int))
    vacsiteInd = superBCC.index(np.zeros(3, dtype=int), (0, 0))[0]
    assert vacsiteInd == 0

    clusexp = cluster.makeclusters(crys, 1.01*a0, MaxOrder)  # Search for 2-body nn clusters
    Tclusexp = cluster.makeclusters(crys, 1.01*a0, MaxOrderTrans)

    VclusExp = Cluster_Expansion.VectorClusterExpansion(superBCC, clusexp, Tclusexp, jnetBCC,
                                                             NSpec, 1, vacsite, MaxOrder,
                                                             MaxOrderTrans)

    print("Done setting up")
    
    # save it for later
    with open("ClusterExpander.pkl", "wb") as fl:
        pickle.dump(VclusExp, fl)

Pre-created Cluster expander found


## Set up the JIT classes for MC and KMC

In [5]:
# First, we have to generate all the arrays
# Lattice gas - set all energies to zero
Energies = np.zeros(len(VclusExp.SpecClusters))
KRAEnergies = [np.zeros(len(val)) for (key, val) in VclusExp.KRAexpander.clusterSpeciesJumps.items()]

numSitesInteracts, SupSitesInteracts, SpecOnInteractSites, Interaction2En, numVecsInteracts,\
VecsInteracts, VecGroupInteracts, numInteractsSiteSpec, SiteSpecInterArray, vacSiteInd,\
InteractionIndexDict, InteractionRepClusDict, Index2InteractionDict, repClustCounter,\
Interact2RepClusArray, Interact2SymClassArray\
= VclusExp.makeJitInteractionsData(Energies)

TsInteractIndexDict, Index2TSinteractDict, numSitesTSInteracts, TSInteractSites, TSInteractSpecs, \
jumpFinSites, jumpFinSpec, FinSiteFinSpecJumpInd, numJumpPointGroups, numTSInteractsInPtGroups, \
JumpInteracts, Jump2KRAEng =\
    VclusExp.KRAexpander.makeTransJitData(KRAEnergies)

Done Indexing interactions : 0.7331323623657227
Done with chemical and symmetry class data for interactions : 0.5700802803039551
Done with vector and energy data for interactions : 1.1408350467681885


In [7]:
# Make the MC class to sample initial states
N_units=8
MCJit = MC_JIT.MCSamplerClass(
    numSitesInteracts, SupSitesInteracts, SpecOnInteractSites, Interaction2En, Interact2RepClusArray,
    Interact2SymClassArray, numVecsInteracts, VecsInteracts, VecGroupInteracts, numInteractsSiteSpec,
    SiteSpecInterArray, numSitesTSInteracts, TSInteractSites, TSInteractSpecs, jumpFinSites, jumpFinSpec,
    FinSiteFinSpecJumpInd, numJumpPointGroups, numTSInteractsInPtGroups, JumpInteracts, Jump2KRAEng
)
# Make the KMC class to generate trajectories
siteIndtoR, RtoSiteInd = VclusExp.makeSiteIndToSite()
KMCJit = MC_JIT.KMC_JIT(numSitesInteracts, SupSitesInteracts, SpecOnInteractSites, Interaction2En,
                        Interact2RepClusArray, Interact2SymClassArray, numVecsInteracts, VecsInteracts,
                        VecGroupInteracts, numInteractsSiteSpec, SiteSpecInterArray, numSitesTSInteracts,
                        TSInteractSites, TSInteractSpecs, jumpFinSites, jumpFinSpec, FinSiteFinSpecJumpInd,
                        numJumpPointGroups, numTSInteractsInPtGroups, JumpInteracts, Jump2KRAEng,
                        siteIndtoR, RtoSiteInd, N_units)

## Set up an initial state

In [10]:
NSpec = 3
state = np.zeros(VclusExp.Nsites, dtype=int)
c0 = 0.75
N0 = int(c0*VclusExp.Nsites)
for i in range(N0):
    state[i] = 0
for i in range(N0, VclusExp.Nsites):
    state[i] = 1

# permute it
state = np.random.permutation(state)
# put the vacancy at the origin
state[vacSiteInd] = NSpec - 1
specs, counts = np.unique(state, return_counts=True)
print("N0:{}, N1:{}, Nvac:{}".format(counts[0], counts[1], counts[2]))

N0:383, N1:128, Nvac:1


In [11]:
# Make the initial state of the trajectory
stateInit = state.copy()
offscInit = MC_JIT.GetOffSite(stateInit, MCJit.numSitesInteracts,
                          MCJit.SupSitesInteracts, MCJit.SpecOnInteractSites)

# Get the site and total cluster symmetry information
statePrintInit, StateTotalSymInit = VclusExp.GetStateSymInfo(stateInit)

In [12]:
# Get the jump site lists
jList, dxList = VclusExp.KRAexpander.ijList, VclusExp.KRAexpander.dxList
jList, dxList

(array([  1,   7, 511,  73,   8,  56,  64, 448]),
 array([[ 0.5,  0.5, -0.5],
        [-0.5, -0.5,  0.5],
        [-0.5, -0.5, -0.5],
        [ 0.5,  0.5,  0.5],
        [ 0.5, -0.5,  0.5],
        [-0.5,  0.5, -0.5],
        [-0.5,  0.5,  0.5],
        [ 0.5, -0.5, -0.5]]))

In [ ]:
# Make the training loop here
Ntraj = 0
Nsteps = 1
Spec = 1 # Species whose diff we are interested in
NswapTrials = 0
swapTrials = np.zeros((initSites.shape[0], 2))

# Copy the initial state information
offscNext = offscInit.copy()
stateNext = stateInit.copy()
siteSymCountsNext = siteSymCountsInit.copy()
symCountsTotalNext = SymCountsTotalInit.copy()

# Set vacancy exchange rates
SpecRates = np.array([0.001, 1.])

# Set up ML hyperparameters here

for traj in range(Ntraj):
        
    # Inputs for first state
    in1 = siteSymCountsNext.copy()
    sym1 = SymCountsTotalNext.copy()
    
    # Extract vector information for state 1 to compute y1
    
    
    # ...
    
    # Compute y1
    # ...
    
    
    X_steps, t_steps, jmpSelectSteps, jmpFinSiteList = KMCJit.LatGasTraj(stateNext, siteSymCountsNext, 
                                                                         symCountsTotalNext, offscNext, 
                                                                         SpecRates, Nsteps, ijList,
                                                                         dxList, vacSiteInd)

    # Translate the final state to bring the vacancy back to the origin
    in2 = siteSymCountsNext.copy()
    sym2 = symCountsTotalNext.copy()
    
    # Extract vector information for state2 to compute y2
    # ...
    # Compute y2
    # ...
    
    # Compute delY, loss and back-propagate
    
    ## Set up the next iteration
    # We have a lattice gas, so all states have equal probability
    # Generate initial state for the next run by randomly permuting stateNext (since offscNext can be
    # easily updated)
    
    # First, translate stateNext so that the vacancy is at vacSiteInd
    
    # Get where the vacancy is now in stateNext
    vacSiteNow = np.where(stateNext == NSpec-1)[0][0]
    # if single step, assert that this is a jump site
    # if Nsteps == 1:
        # assert vacSiteNow in ijList
        # assert vacSiteNow == ijList[jmpSelectSteps[0]]
    
    stateNext = KMCJit.TranslateState(stateNext, vacSiteInd, vacSiteNow, siteSymCountsNext)
    assert stateNext[vacSiteInd] == NSpec - 1
    
    # Gather the random swaps
    stateSwap = stateNext.copy()
    initSites, finSites = MC_JIT.DoRandSwap(stateSwap, NswapTrials, vacSiteInd)
    
    # Now also concatenate the permutation of jump site occupancies
    initSites = np.concatenate((initSites, jList))
    finSites = np.concatenate((finSites, np.random.permuation(jList)))
    
    swapTrials[:, 0] = initSites
    swpaTrials[:, 1] = finSites
    
    # Now update stateNext for the next trajectory
    MCJit.GetNewRandState(stateNext, offScNext, siteSymCountsNext, symCountsTotalNext,
                          swapTrials, 0.0)

In [23]:
MCJit.numInteractsSiteSpec[0, 2]

173

In [38]:
nums = set()
for site in range(VclusExp.Nsites):
    for spec in range(NSpec):
        nums.add((MCJit.numInteractsSiteSpec[site, spec], spec))

In [39]:
nums

{(173, 2), (331, 0), (331, 1)}